In [ ]:
import os
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"

In [ ]:
import subprocess
import gc
import json
from datetime import datetime
from tqdm import tqdm

# -------- LOAD HF TOKEN FROM DATASET ---------
HF_TOKEN_PATH = "/kaggle/input/imggenhub-hf-token/hf_token.json"
with open(HF_TOKEN_PATH, "r") as f:
    HF_TOKEN = json.load(f)["HF_TOKEN"]
os.environ["HF_TOKEN"] = HF_TOKEN
print("HF_TOKEN loaded from dataset")

# -------- SETTINGS ---------
# Choose model source: "dataset" or "huggingface"
MODEL_SOURCE = "huggingface"

# CLI-configurable model parameters with defaults
DIFFUSION_REPO_ID = "city96/FLUX.1-schnell-gguf"
DIFFUSION_FILENAME = "flux1-schnell-Q4_0.gguf"
VAE_REPO_ID = "black-forest-labs/FLUX.1-schnell"
VAE_FILENAME = "ae.safetensors"
CLIP_L_REPO_ID = "comfyanonymous/flux_text_encoders"
CLIP_L_FILENAME = "clip_l.safetensors"
T5XXL_REPO_ID = "comfyanonymous/flux_text_encoders"
T5XXL_FILENAME = "t5xxl_fp8_e4m3fn.safetensors"

if MODEL_SOURCE == "dataset":
    # Model paths from Kaggle datasets
    DIFFUSION_MODEL = "/kaggle/input/flux1-schnell-q4-zip/flux1-schnell-Q4_0.gguf"
    VAE_MODEL = "/kaggle/input/vae-zip/ae.safetensors"
    CLIP_L_MODEL = "/kaggle/input/clip-l-zip/clip_l.safetensors"
    T5XXL_MODEL = "/kaggle/input/t5xxl-zip/t5xxl_fp8_e4m3fn.safetensors"
else:
    # Download from HuggingFace using configurable repo/filename
    from huggingface_hub import hf_hub_download
    print("Downloading models from HuggingFace...")
    
    DIFFUSION_MODEL = hf_hub_download(
        repo_id=DIFFUSION_REPO_ID,
        filename=DIFFUSION_FILENAME,
        token=HF_TOKEN
    )
    VAE_MODEL = hf_hub_download(
        repo_id=VAE_REPO_ID,
        filename=VAE_FILENAME,
        token=HF_TOKEN
    )
    CLIP_L_MODEL = hf_hub_download(
        repo_id=CLIP_L_REPO_ID,
        filename=CLIP_L_FILENAME,
        token=HF_TOKEN
    )
    T5XXL_MODEL = hf_hub_download(
        repo_id=T5XXL_REPO_ID,
        filename=T5XXL_FILENAME,
        token=HF_TOKEN
    )
    print("Models downloaded successfully!")

# -------- PARAMETERS (can be overridden when running via papermill) ---------
PROMPTS = ['Fresh flux gguf test']
NEGATIVE_PROMPT = ""
OUTPUT_DIR = "."
IMG_SIZE = (512, 512)
GUIDANCE = 0.8
STEPS = 1
SEED = 42
PRECISION = "q4"
# ----------------------------

print(f"Device: GPU (FLUX GGUF)")
print(f"Model Source: {MODEL_SOURCE}")
print(f"\nDiffusion Model: {DIFFUSION_REPO_ID}/{DIFFUSION_FILENAME}")
print(f"VAE Model: {VAE_REPO_ID}/{VAE_FILENAME}")
print(f"CLIP-L Model: {CLIP_L_REPO_ID}/{CLIP_L_FILENAME}")
print(f"T5XXL Model: {T5XXL_REPO_ID}/{T5XXL_FILENAME}")
print(f"\nPrompts: {len(PROMPTS)}")
print(f"Precision: {PRECISION}")
print(f"Steps: {STEPS}")
print(f"Image size: {IMG_SIZE}")
print(f"Output directory: {OUTPUT_DIR}")

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Setup stable-diffusion.cpp executable
!mkdir -p /kaggle/working/stable-diffusion.cpp
!cp -r /kaggle/input/sd-build-zip/kaggle/working/export/sd_build/* /kaggle/working/stable-diffusion.cpp/
!chmod +x /kaggle/working/stable-diffusion.cpp/build/bin/sd

sd_executable = "/kaggle/working/stable-diffusion.cpp/build/bin/sd"
if not os.path.isfile(sd_executable):
    raise FileNotFoundError("sd executable not found at: " + sd_executable)

print(f"SD executable ready: {sd_executable}")

In [ ]:
# Generate images for each prompt
print("-"*60)
print(f"Model paths:")
print(f"  Diffusion: {DIFFUSION_MODEL}")
print(f"  VAE: {VAE_MODEL}")
print(f"  CLIP-L: {CLIP_L_MODEL}")
print(f"  T5XXL: {T5XXL_MODEL}")
print("-"*60)

for i, prompt in enumerate(tqdm(PROMPTS, desc="Generating")):
    print("-"*60)
    print(f"Prompt {i+1}/{len(PROMPTS)}: {prompt[:60]}...")
    print("-"*60)
    
    # Generate unique filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"generated_{i+1}_{timestamp}.png"
    output_path = os.path.join(OUTPUT_DIR, filename)
    
    cmd = [
        sd_executable,
        "--diffusion-model", DIFFUSION_MODEL,
        "--vae", VAE_MODEL,
        "--clip_l", CLIP_L_MODEL,
        "--t5xxl", T5XXL_MODEL,
        "--prompt", prompt,
        "--cfg-scale", str(GUIDANCE),
        "--sampling-method", "euler",
        "--steps", str(STEPS),
        "--width", str(IMG_SIZE[1]),
        "--height", str(IMG_SIZE[0]),
        "--seed", str(SEED),
        "--output", output_path,
        "--rng", "cuda",
        "-v",
    ]
    
    print(f"Running generation...")
    res = subprocess.run(cmd, capture_output=True, text=True)
    
    print(f"Return code: {res.returncode}")
    if res.returncode == 0:
        print(f"Saved: {output_path}")
    else:
        print(f"ERROR: Generation failed")
        print(f"STDOUT: {res.stdout}")
        print(f"STDERR: {res.stderr}")
    
    # Cleanup
    gc.collect()

print("-"*60)
print(f"COMPLETE! {len(PROMPTS)} images saved to {OUTPUT_DIR}")
print("-"*60)